In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
import warnings

warnings.filterwarnings( 'ignore' )

In [3]:
train_data = pd.read_csv("train.csv") 
test_data = pd.read_csv("test.csv")
sample_result = pd.read_csv("sample_submission.csv")